In [1]:
import collections
import bisect

## sliding window max

In [2]:
"""
Simple solution O(kn) best case and O(n**2) worst case
"""

def maxSlidingWindow(a,k):
    res = []
    for i in range(len(a)):
        if i + k <= len(a):
            res.append(max(a[i:i + k]))
    return res


"""
    O(n) solution
"""

def maxSlidingWindow(nums, k):
    d = collections.deque()
    out = []
    for i, n in enumerate(nums):
        
        """
        The below statement insures that d[0] will always be maximum
        """
        while d and nums[d[-1]] < n:
            d.pop()
        d += i,
        
        """
        with above statement d may have values less than d[0] in the array
        Below statements remove d[0] value if max length required is reached.
        """
        if d[0] == i - k:
            d.popleft()
            
        """
        For each iteration d[0] will be maximum value.
        Here we add those values to the output list 
        if the index is greadter or equal to k-1 
        i.e. we igone first k-1 iterations.
        """
        if i >= k - 1:
            out += nums[d[0]],
    return out

In [3]:
maxSlidingWindow([1,3,2,4,5], 3)

[3, 4, 5]

## sliding window median

In [4]:
"""
The below is O(nlogk) solution which also can be used for sliding window max prolblem

The time complexity of bisect.biscet and biscet.insort is O(log K) we do this for n times 
so total time complexity is O(N log K)

"""

def medianSlidingWindow(nums, k):
    """
    First we get the first window and sort it
    """
    win, rv = nums[:k], []
    win.sort()
    
    """
    the below step will be useful for median calculation
    """
    odd = k%2
    
    for i,n in enumerate(nums[k:],k):
        
        """
        depending upon the even odd we apply median calculation to our window and append it to result
        """
        rv.append((win[k/2]+win[k/2-1])/2. if not odd else win[(k-1)/2]*1.)
        
        """
        we pop the element which will go out of window
        we can get that element using indices in nums, 
        next we determine index of that element in window and pop it.
        """
        win.pop(bisect.bisect(win, nums[i-k])-1) # <<< bisect is faster than .remove()
        
        """
        we insert the new element and keep the window sorted.
        """
        bisect.insort(win, n)
    rv.append((win[k/2]+win[k/2-1])/2. if not odd else win[(k-1)/2]*1.)
    return rv

"""
check two heap solution as well.
"""

'\ncheck two heap solution as well.\n'

In [5]:
medianSlidingWindow([1,3,-1,-3,5,3,6,7], 3)

[1.0, -1.0, -1.0, 3.0, 5.0, 6.0]

## find median of data stream

In [6]:
"""
The idea is to have a min heap and a max heap
by default heapq data structure in python is min heap
to have a max heap data structure we can add -ve of number to the heap 
and with pop operation we can again do -ve
"""

class MedianFinder:
    def __init__(self):
        self.small = []
        self.large = []

    def addNum(self, num):
        if len(self.small) == len(self.large):
            heappush(self.large, -heappushpop(self.small, -num))
        else:
            heappush(self.small, -heappushpop(self.large, num))

    def findMedian(self):
        if len(self.small) == len(self.large):
            return float(self.large[0] - self.small[0]) / 2.0
        else:
            return float(self.large[0])
        
        
"""
solution using bisect
"""

class MedianFinder:
    def __init__(self):
        self.array = []
        self.count = 0

    def addNum(self, num):
        bisect.insort(self.array, num)
        self.count += 1

    def findMedian(self):
        if self.count == 0:
            return 0
        elif self.count % 2 != 0:
            return self.array[self.count//2] * 1.0
        else:
            return (self.array[self.count//2] + self.array[self.count//2 -1]) / 2.0

## Paliandrome pairs

In [7]:
"""
Naive solution is to compare every word with every other word
Time complexity o(n**2)
"""

def palindromePairs(words):
    """
    :type words: List[str]
    :rtype: List[List[int]]
    """
    words_rev = [word[::-1] for word in words]

    ans = []
    for index1 in range(len(words)):
        for index2 in range(len(words)):
            if index1 == index2:
                continue
            word_1 = words[index1]
            word_2 = words[index2]

            word_1_rev = words_rev[index2]
            word_2_rev = words_rev[index1]
            if word_1 + word_2 == word_1_rev + word_2_rev:
                ans.append((index1, index2))
    return ans


"""
Best solution is to do comparisions efficiently.
We will generate a data structure such that 

    dictionary:
    its keys is length of the squence 
    its value is dictionary of word and its index


    Step 1: To check if the reverse of the string is present or not
        for that we will iterate through the dictionary keys 
        and for each word in dictionary[key] we will check if reverse of the word is present in the dictionary or not
        due to dictionary we can search in constant time
        
    Step 2: To check if part of strings is present in the data structure and if present it creates a paliandrome.
        We will iterate through all the keys in the dictionary 
        for each key we check if start substring or end-substring with length less than key is present in dictionary or not.
    
"""

"""
Input : ["abcd","dcba","lls","s","sssll"]

dictionary data structure: {1: {'s': 3}, 3: {'lls': 2}, 4: {'dcba': 1, 'abcd': 0}, 5: {'sssll': 4}}


"""

def palindromePairs(words):
        """
        :type words: List[str]
        :rtype: List[List[int]]
        """
        
        dic={}
        for i,wd in enumerate(words):
            if len(wd) in dic:
                dic[len(wd)][wd]=i
            else:
                dic[len(wd)]={wd:i}
        ans=[]
        # same length
        for k in dic:
            for wd in dic[k]:
                if wd[::-1] in dic[k]:
                    i=dic[k][wd]
                    j=dic[k][wd[::-1]]
                    if i!=j:
                        ans.append([i,j])
        # diff length
        for k in dic:
            for l in range(k):
                if l in dic:
                    for wd in dic[k]:
                        left=wd[:-(l+1):-1]
                        right=wd[:l][::-1]
                        if left in dic[l]:
                            leftwd=left+wd
                            if leftwd==leftwd[::-1]:
                                ans.append([dic[l][left],dic[k][wd]])
                        if right in dic[l]:
                            rightwd=wd+right
                            if rightwd==rightwd[::-1]:
                                ans.append([dic[k][wd],dic[l][right]])
        return ans

In [8]:
palindromePairs(["abcd","dcba","lls","s","sssll"])

[[0, 1], [1, 0], [3, 2], [2, 4]]

## merge k sorted lists

In [9]:
"""
using priority queue

put take log k time 

overall time O(n log k)
"""

from Queue import PriorityQueue
def mergeKLists(lists):
    q = PriorityQueue()
    for ll in lists:
        if ll:
            q.put((ll.val, ll))
    head = ListNode(0)
    dummy = head
    while not q.empty():
        value, node = q.get()
        head.next = ListNode(value)
        head = head.next
        if node:
            q.put((node.val, node))
    return dummy.next


"""
another solution is directly combining lists and using sort

sorting takes O(n log n) time 

total time complexity O(n log n)
"""
def mergeKLists(lists):
    arr = []
    nodes = []
    for ll in lists:
        while ll:
            arr.append(ll.val)
            nodes.append(ll)
            ll = ll.next
    arr.sort()
    length = len(arr)
    i = 0
    for elem in arr:
        nodes[i].val = elem
        if i+1 < length:
            nodes[i].next = nodes[i+1]
        else:
            nodes[i].next = None
        i += 1
    if len(nodes)>0:
        return nodes[0]

    return None

## min cost to Paint House : 3 cost array

In [10]:
"""
The operations must be done inline or using temp variable

RunTime : O(n)
"""
def minCost(self, costs):
        cost_red, cost_blue, cost_green = 0,0,0
        for red, blue, green in costs:
            cost_red, cost_blue, cost_green = red + min(cost_blue, cost_green), blue + min(cost_red, cost_green), green + min(cost_red, cost_blue)
        return min(cost_red, cost_blue, cost_green)


## min cost to Paint House : cost array length undefined

In [11]:
"""
Here we modify the array in place
if that is not permitted we can assign O(n) extra space and perform the operations

We calculate two minimums from previous iteration 
we add min1 to current iteration value if its index is not equal to min1 index else we add min2 
"""


def minCostII(costs):
    
    if not costs:
        return 0
    
    num_costs = len(costs)
    len_costs = len(costs[0])
    
    for i in range(1, num_costs):
        min1 = min(costs[i-1])
        index = costs[i-1].index(min1)
        min2 = min(costs[i][:index] + costs[i][index+1:])
        
        for j in range(len_costs):
            if j == index:
                costs[i][j] += min2
            else:
                costs[i][j] += min1
                
    return min(costs[-1])

## Burst Balloons

In [12]:
"""
Solution with memo
"""

class Solution:
    def maxCoins(self, nums):
        memo, nums = {}, [1] + [num for num in nums if num] + [1]
        print(nums)
        def dfs(l, r):
            if r - l == 1: return 0
            if (l, r) not in memo: memo[(l, r)] = max(nums[l] * nums[i] * nums[r] + dfs(l, i) + dfs(i, r) for i in range(l + 1, r))
            print(memo)
            return memo[(l, r)]
        return dfs(0, len(nums) - 1)
    
    
"""
Solution with dp
RunTime = O(n ** 2)
"""

class Solution:
    def maxCoins(self, nums):
        nums = [1] + nums + [1]
        n = len(nums)
        dp = [[0]*n for _ in range(n)]
        for gap in range(2,n):
            for i in range(n-gap):
                j = i + gap
                product = nums[i] * nums[j]
                dp[i][j] = max(product * nums[k] + dp[i][k] + dp[k][j] for k in range(i+1,j))
        return dp[0][n-1]     


## K empty slots

In [13]:
def kEmptySlots(flowers, k):
    blomming = []
    for day, position in enumerate(flowers, 1):
        index = bisect.bisect(blomming, position)
        for i in range(index - index>0, index +1):
            if abs(position - i) -1 == k:
                return day
        bisect.insort(blomming, position)
    return -1

## Couple holding hands

In [14]:
"""
Greedy solution iterate through the array 2 by 2 elements
"""

class Solution(object):
    def minSwapsCouples(self, row):
        ans = 0
        for i in xrange(0, len(row), 2):
            x = row[i]
            if row[i+1] == x^1: continue
            ans += 1
            for j in xrange(i+1, len(row)):
                if row[j] == x^1:
                    row[i+1], row[j] = row[j], row[i+1]
                    break
        return ans

## additive num

In [15]:
"""

Input: "112358"
Output: true 
Explanation: The digits can form an additive sequence: 1, 1, 2, 3, 5, 8. 
             1 + 1 = 2, 1 + 2 = 3, 2 + 3 = 5, 3 + 5 = 8
             
"""

def isAdditiveNum(n):
    n = len(nums)
    """
    iterate such that 
    
    for i in range(1,n):
        for j in range(i+1,n):

    """
    for i, j in itertools.combinations(range(1,n), 2):
        a, b = nums[:i], nums[i:j]
        
        """
        to check whether the number starts with zeros
        """
        if b != str(int(b)):
            continue
        while j < n:
            """
            we iteratively compute whether the next number is present in nums at location j
            if present we continue the loop for next numbers
            """
            
            c = str(int(a) + int(b))
            if not nums.startswith(c, j):
                break
            j += len(c)
            a, b = b, c
        if j == n:
            return True
    return False

## ones and zeros

In [16]:
"""
Input: Array = {"10", "0001", "111001", "1", "0"}, m = 5, n = 3
Output: 4
"""

"""
create a array with count of zeros and ones for each item in the list

Here we want to find maximum number of items we can select from list such that sum of zero < 5 and sum one < 3

so, we can use greedy approach here 
    sort the numbers with number of zeros
    start from index pick elements such that num zero/ one are less than the required count

"""


def findMaxForm(self, strs, m, n):
    def count(s):
        ones=zeros=0
        for x in s:
            if x=='0':
                zeros+=1
            else:
                ones+=1
        return zeros, ones

    ans=0
    L=len(strs)
    matrix=[count(s) for s in strs]
    matrix.sort(key= lambda x: x[1])
    for i in range(L):
        zeros, ones = matrix[i]
        if ones > n or zeros>m:
            continue
        cur=1
        for j in range(i+1,L):
            z,o=matrix[j]
            if ones+o<= n and zeros+z<=m:
                ones+=o
                zeros+=z
                cur+=1
        ans=max(ans,cur)

    return ans

"""
DP solution : Knapsack O(length * m * n) time, O(mn) space
"""



'\nDP solution : Knapsack O(length * m * n) time, O(mn) space\n'

## median of two sorted arrays

In [17]:
def findMedianOfTwoSorted(nums1, nums2):
    l1 = len(nums1)
    l2 = len(nums2)
    total = l1 + l2
    if total % 2 :
        return (findKthSmallest(nums1, nums2, total//2) + findKthSmallest(nums1, nums2, total//2 + 1)) / 2.0
    else:
        return findKthSmallest(nums1, nums2, total//2)

def findKthSmallest(nums1, nums2, k):
    if len(nums1) > len (nums2):
        findMedianOfTwoSorted(nums2, nums1)
    if not nums1:
        return nums2[k:k+1]
    if k == 0:
        return min(nums1[0], nums2[0])
    
    
    ## reduce step
    indexa = min(k//2, len(nums1))
    indexb = len(nums2) - indexa
    
    ## reduce a if
    if nums1[indexa] <= nums2[indexb]:
        findKthSmallest(nums1[indexa:], nums2, k - indexa)
    else:
        findKthSmallest(nums1, nums2[indexb:], k - indexb)
            

## Construct a tree from a strng with parenthesis

In [18]:
"""
Input: "4(2(3)(1))(6(5))"
Output: return the tree root node representing the following tree:
"""


"""
Questions to ask:
    1. is the tree balanced/ binary ?
    2. does the string contains -ve numbers
    3. can i assume that i am getting perfect input, with matched parenthesis
"""


def str2tree(self, s):
    """
        The idea is to find left subtree parenthesis and right subtree parenthesis and process them recursively
        Time conplexity: roughtly O(n**2) as for each node we try to find middle index.
    """
    def findIndex(s, start, end):
        stack = []
        index = -1
        for ch in s[start:end]:
            index += 1
            if ch == '(':
                stack.append(ch)
            elif ch == ')':
                if stack[-1] == '(':
                    stack.pop()
                if not stack:
                    return start + index
        return None

    def st2t(s, start, end):
        if start > end:
            return None
        negative = 0
        if s[start] == "-":
            negative = 1
        root = TreeNode(s[start:start+1+negative]) 
        index = findIndex(s, start+1+negative, end)
        if index:
            root.left = st2t(s, start+2 + negative, index)
            root.right = st2t(s, index+2, end-1)
        return root
    if not s: return None
    return st2t(s, 0, len(s))

## solution 2

def str2tree(self, s):
        """
       The idea is to construct the tree while traversing the string
       RunTime : O(n)
           memory : O(n)
       
       
       push pop oprations for the node and parenthesis are done simulteniously.
        """
        if s == '':
            return None
        stack = []
        num = ''

        for c in s:
            if c.isdigit() or c == '-':
                num += c
            elif num:
                node = TreeNode(int(num))
                if not stack:
                    root = node
                elif num:
                    if not stack[-1].left:
                        stack[-1].left = node
                    else:
                        stack[-1].right = node
                if c == '(':
                    stack.append(node)
                num = ''
            elif c == ')':
                stack.pop()
        return root if num == '' else TreeNode(int(num))

## kadane's algorithm

In [19]:
#Kadane's algorithm
def kadane(A):
    ans = cur = None
    for x in A:
        cur = x + max(cur, 0)
        ans = max(ans, cur)
    return ans

## kadane for circular array

In [20]:
def maxSubarraySumCircular(self, A):
        # ans1: answer for one-interval subarray
        ans1 = cur = None
        for x in A:
            cur = x + max(cur, 0)
            ans1 = max(ans1, cur)

        # ans2: answer for two-interval subarray, interior in A[1:]
        ans2 = cur = float('inf')
        for i in xrange(1, len(A)):
            cur = A[i] + min(cur, 0)
            ans2 = min(ans2, cur)
        ans2 = sum(A) - ans2

        # ans3: answer for two-interval subarray, interior in A[:-1]
        ans3 = cur = float('inf')
        for i in xrange(len(A)-1):
            cur = A[i] + min(cur, 0)
            ans3 = min(ans3, cur)
        ans3 = sum(A) - ans3

        return max(ans1, ans2, ans3)

## Partition to K Equal Sum Subsets

In [21]:
"""correct mistakes in solution"""

def partition_k_subsets(array, parts):
    length = len(array)
    visited = [0] * length
    if parts <= 1: return True
    
    div, mod = divmod(sum(array), parts)
    if mod: return False
    array.sort(reverse=True)
    count = 0
    
    def dfs(start,s, k):
        if s == div:
            dfs(0,0, k-1)
        elif k == 1:
            return True
        for index in range(start, length):
            if not visited[index] and  s+array[index] <= div:
                visited[index] = True
                if dfs(index+1, s+array[index], k):
                    return True
                visited[index] = False
        return False
    return dfs(0, 0, parts)

partition_k_subsets([1,2,3],2)

False

## max profit

    keep min_profit while iterating array and calculate max profit at each step

## Search in rotated sorted array

In [22]:
def searchRotated(nums, ele):
    if not nums:
        return -1
    
    def binarySearch(start, end):
        if start > end:
            return -1
        mid = (start + end) // 2
        if nums[mid] == ele:
            return mid
        elif nums[start] <= target <= nums[mid]:
            return binarySearch(start, mid-1)
        elif nums[mid] <= target <= nums[end]:
            return binarySearch(mid+1, end)
        elif nums[mid] > nums[end]:
            return binarySearch(mid+1, end)
        elif nums[start] > nums[mid]:
            return binarySearch(start, mid-1)
        
    return binarySearch(0, len(nums)-1)



"""
If the array has duplicate we will remove them at start of each iteration
"""

def search(self, nums, target):
    if not nums:
        return False

    def binarySearch(start, end):
        if start > end:
            return False
        while start < end and nums[start] == nums[start+1]:
            start += 1
        while end > start and nums[end] == nums[end-1]:
            end -= 1
        mid = (start + end) // 2
        if nums[mid] == target:
            return True
        elif nums[start] <= target <= nums[mid]:
            return binarySearch(start, mid-1)
        elif nums[mid] <= target <= nums[end]:
            return binarySearch(mid+1, end)
        elif nums[mid] > nums[end]:
            return binarySearch(mid+1, end)
        elif nums[start] > nums[mid]:
            return binarySearch(start, mid-1)
        return False

    return binarySearch(0, len(nums)-1)

## minimum in rotated array
    as array is rotated minimum must be in rotated part

In [23]:
"""
if the array is sorted we will reduce array from start and end
"""

def findMin(self, nums):
    start = 0
    end = len(nums) - 1
    while start < end - 1:
        while start < end -1 and nums[start] == nums[start + 1]:
            start += 1
        while start < end -1 and nums[end] == nums[end - 1]:
            end -= 1
        mid = (start + end) // 2
        if nums[mid] > nums[end]:
            start = mid
        else:
            end = mid
    return min(nums[start], nums[end])

## sorted array to BST

In [24]:
"""
The  idea is similar to the preorder traversal
"""

def sortedArrayToBST(self, num):
    if not num:
        return None

    mid = len(num) // 2

    root = TreeNode(num[mid])
    root.left = self.sortedArrayToBST(num[:mid])
    root.right = self.sortedArrayToBST(num[mid+1:])

    return root


"""
Sorted list to BST
"""

def sortedListToBST(self, head):
    def findMiddle(head):
        if not head:
            return None
        slow = fast = head
        prev = None
        while fast and fast.next:
            prev = slow
            slow = slow.next
            fast = fast.next.next
        return prev

    def build(head):
        if head is None:
            return None
        if head.next is None:
            return TreeNode(head.val)
        mid_prev = findMiddle(head)
        mid = mid_prev.next
        mid_next = mid.next
        mid_prev.next = None
        mid.next = None
        node = TreeNode(mid.val)
        node.left = build(head)
        node.right = build(mid_next)
        return node
    return build(head)

## Max Chunks To Make Sorted

In [25]:
"""
Input: arr = [4,3,2,1,0]
Output: 1
Explanation:
Splitting into two or more chunks will not return the required result.
For example, splitting into [4, 3], [2, 1, 0] will result in [3, 4, 0, 1, 2], which isn't sorted.
"""

def maxChunksToSorted(self, arr):
    ans = ma = 0
    for i, x in enumerate(arr):
        ma = max(ma, x)
        if ma == i: ans += 1
    return ans


"""
with duplicates
"""
def maxChunksToSorted(self, arr):
    count = collections.defaultdict(int)
    ans = nonzero = 0

    for x, y in zip(arr, sorted(arr)):
        count[x] += 1
        if count[x] == 0: nonzero -= 1
        if count[x] == 1: nonzero += 1

        count[y] -= 1
        if count[y] == -1: nonzero += 1
        if count[y] == 0: nonzero -= 1

        if nonzero == 0: ans += 1

    return ans

## remove duplicates

In [26]:
def removeDuplicates(nums):
    if not nums: return 0
    index = 0
    for i in range(1,len(nums)):
        if nums[i] != nums[index]:
            index += 1
            nums[index] = nums[i]
    return index + 1

## Best meeting point 

In [27]:
"""
Give a grid with points and no obstacle find best meeting point
"""

"""
The idea is to store all the row columns value for the points and the best meeting point will be the middle of the 
row and column sorted

Find distance from that point to all point.
"""

def minTotalDistance(self, grid):
        
        if not grid:
            return 0, 0
        horizontal = []
        vertical = []
        
        row_len = len(grid)
        col_len = len(grid[0])
        points = []
        count = 0
        
        
        for row in range(row_len):
            for col in range(col_len):
                if grid[row][col] == 1:
                    count += 1
                    points.append((row,col))
                    horizontal.append(row)
                    vertical.append(col)
        
        horizontal.sort()
        vertical.sort()
        meet_x, meet_y = horizontal[count//2], vertical[count//2]
        result = 0
        
        for x, y in points:
            result += abs(meet_x-x) + abs(meet_y-y)
            
        return result

## number of binary search trees

In [28]:
## for any increasing numbers no. of binary search trees are same.
## 1,2,3 will have same number of bst as 5,6,7

## to find number of binary search trees for n
## iterate through the array with each array element as a root.
## multiply number of left sub trees with number of right subtrees

def numTrees(self, n):
    res = [0 for i in range(n+1)]
    res[0] = 1
    res[1] = 1

    for i in range(2,n+1):
        for j in range(0,i):
            res[i] += res[j] * res[i-j-1]

    return res[n]

## generate binary trees

In [29]:
"""
nums :- integer
"""

def generate(nums):
    def node(val, left, right):
        node = TreeNode(val)
        node.left = left
        node.right = right
        return node
    def trees(nums, start, end):
        return [node(index, left, right) for index in range(start, end) for left in trees(start, index-1) for right in range(index+1, end)] or [None]
    return trees(0, len(nums)+1)

## Imp** Longest Paliandromic subsequence

In [30]:
string = "abcda"
ans = "abcba"
def longestPalindromic(string):
    length = len(string)
    ret = [[0 for i in range(length)] for j in range(length)]
    for i in range(length):
        ret[i][i] = 1
    for gap in range(1, length):
        for row in range(length-gap):
            col = row + gap
            if gap == 1 and string[row] == string[col]:
                ret[row][col] = 2
            else:
                if string[row] == string[col]:
                    ret[row][col] = 2 + ret[row+1][col-1]
                else:
                    ret[row][col] = max(ret[row-1][col], ret[row][col-1])
    return ret[0][length-1]
longestPalindromic(string)

3

## Cutting Rod

In [31]:
def cut_rod(prices):
    length = len(prices)
    ret = [[0 for _ in range(length+1)] for _ in range(length+1)]
    for row in range(1, length+1):
        for col in range(1, length+1):
            if col < row:
                ret[row][col] = ret[row-1][col]
            else:
                ret[row][col] = max(ret[row][col-1], ret[row][col-prices[row-1]]+prices[row-1])
    return ret[length][length]
cut_rod([2,3,7,8])

10

## matrix chain multiplication

In [32]:
def matrix_chain(array):
    length = len(array)
    ret = [[0 for _ in range(length)] for _ in range(length)]
    
    
    for gap in range(2, length):
        for row in range(0,length-gap):
            col = row + gap
            ret[row][col] = float('inf')
            for k in range(row+1, col):
                ret[row][col] = min(ret[row][col], ret[row][k] + ret[k][col] + array[row]*array[col]*array[k])
    return ret[0][-1]

matrix_chain([2,3,4,5])

64

## coin change - min coins

In [33]:
def coin_change_min_coin(arr, target):
    row_len = len(arr) + 1
    col_len = target + 1
    
    ret = [[float('inf') for _ in range(col_len)] for _ in range(row_len)]
    
    for row in range(row_len):
        ret[row][0] = 0
    
    for row in range(1, row_len):
        for col in range(1, col_len):
            if col < arr[row-1]:
                ret[row][col] = ret[row-1][col]
            else:
                ret[row][col] = min(ret[row-1][col], ret[row][col-arr[row-1]] + 1)
    print(ret)
    return ret[row_len-1][col_len-1]

coin_change_min_coin([1,2,3], 5)

[[0, inf, inf, inf, inf, inf], [0, 1, 2, 3, 4, 5], [0, 1, 1, 2, 2, 3], [0, 1, 1, 1, 2, 2]]


2

## coin change - num ways

In [34]:
def coin_change_num_ways(coins, target):
    
    if not coins:
        return 0

    ret = [[0 for _ in range(target+1)] for _ in range(len(coins)+1)]
    for row in range(len(coins)+1):
        ret[row][0] = 1
    
    for row in range(1, len(coins)+1):
        for col in range(1,target+1):
            if coins[row-1] > col:
                ret[row][col] = ret[row][col-1]
            else:
                ret[row][col] = ret[row-1][col] + ret[row][col-coins[row-1]]
    return ret[len(coins)][target]

coin_change_num_ways([1,2,3], 5)

4

## 0 1 knapsack

## subset sum problem

## Longest Commmon subsequence/ substring

In [35]:
def longest_common_subsequnce(string1, string2):
    row_len = len(string1) + 1
    col_len = len(string2) + 1
    
    ret = [[0 for _ in range(row_len)] for _ in range(col_len)]
    
    for row in range(1, row_len):
        for col in range(1, col_len):
            if string1[row-1] == string2[col-1]:
                ret[row][col] = ret[row-1][col-1] + 1
            else:
                ret[row][col] = max(ret[row-1][col], ret[row][col-1])
    return ret[-1][-1]

longest_common_subsequnce("abcd", "axbd")

3

## Longest increasing subsequence

In [36]:
def longest_increasing(array):
    length = len(array)
    ret = [1] * length
    
    for i in range(1, length):
        ret[i] = max([ret[j] + 1 for j in range(i) if array[j] <= array[i]]+[1])
    return ret[-1]

longest_increasing([1,3,2,4,5,0,7])

5

## remove K digit such that the new number is minimum

In [37]:
"""
brute force
runtime: exponential
"""

def removeKdigits(self, num, k):
    if len(num) <= k:
        return "0"
    if not k:
        return num
    def removeK(nums, k):
        if not nums:
            return 0
        if not k:
            return int(nums)
        return min(removeK(nums[:i] + nums[i+1:], k-1) for i in range(len(nums)))
    return str(removeK(num, k))

"""
greedy
O(n)
"""

def removeKdigits(num, k):
    stack = []
    nums = list(num)
    for n in nums:
        while stack and n < stack[-1] and k > 0:
            stack.pop()
            k -= 1
        stack.append(n)
    for _ in range(k): stack.pop()
    if not stack: stack.append("0")
    return str(int("".join(stack)))

removeKdigits("19298", 2)

'128'

## string interleaving

In [38]:
"""
dynamic programming
"""

def string_interleaving(str1, str2, str3):
    l1, l2, l3 = len(str1), len(str2), len(str3)
    if l1 + l2 != l3:
        return False
    
    ret = [[False for _ in range(l1+1)] for _ in range(l2+1)]
    # row = str2
    # col = str1
    
    for row in range(l2+1):
        for col in range(l1+1):
            index = row + col - 1
            if row == 0 and col == 0:
                ret[row][col] = True
                
            elif row == 0 and str1[col-1] == str3[index]:
                ret[row][col] = True and ret[row][col-1]
                
            elif col == 0 and str2[row-1] == str3[index]:
                ret[row][col] = True and ret[row-1][col]
                
            else:
                ret[row][col] = (ret[row-1][col] if str2[row-1] == str3[index] else False) or (ret[row][col-1] if str1[col-1] == str3[index] else False)
    
    return ret[-1][-1]

string_interleaving("afd", "sgh", "asgfdh")

True

## Binary Tree Maximum Path Sum

In [39]:
def maxPathSum(self, root):
    longest = [float('-inf')]
    def pathSum(root):
        if root is None:
            return 0
        rootval = root.val
        left = pathSum(root.left)
        right = pathSum(root.right)
        longest[0] = max(longest[0], rootval, rootval+left+right, rootval+left, rootval+right)
        return max(rootval+left, rootval+right, rootval, 0)
    pathSum(root)
    return longest[0]

"""
Improvements : improve condit
"""

'\nImprovements : improve condit\n'

## Maximum consecative ones

In [40]:
"""
we can flip one zero to one
"""

def findMaxConsecutiveOnes(self, nums):
    prev, curr, maxlen = -1, 0, 0
    for n in nums:
        if n == 0:
            prev, curr = curr, 0
        else:
            curr += 1
            maxlen = max(maxlen, prev+curr+1)
    return max(maxlen, prev+curr+1)

## Max distance  to closet person

In [41]:
"""
insert element such  that it is at maximum  distance from the person sitting.

Input: [1,0,0,0,1,0,1]
Output: 2

"""

def maxDistToClosest(self, seats):
    l,r = 0, 0
    ret = 0
    while r < len(seats):
        if l == 0 and seats[l] == 0:
            ret = max(ret, r-l)
        if r == len(seats) - 1 and seats[r] == 0:
            ret = max(ret, r-l)
        if seats[r] == 1:
            ret = max(ret, (r-l)/2)
            l = r
            r += 1
        else:
            r += 1
    return ret


## Exam Room

## Maximum Length of Pair Chain

    Input: [[1,2], [2,3], [3,4]]
    Output: 2
    Explanation: The longest chain is [1,2] -> [3,4]

In [42]:
"""
Gives TLE
"""

def findLongestChain(pairs):
    pairs.sort()
    length = len(pairs)
    
    dp = [1] * length
    
    for i in range(length):
        for j in range(i):
            if pairs[j][1] < pairs[i][0]: 
                dp[i] = max(dp[i], dp[j]+1)
    return max(dp)
            
"""
sort w.r.t end time and find max length pairs
"""

def findLongestChain(pairs):
    pairs.sort(key=lambda x: x[1])
    count = 0
    end = float('-inf')
    for pair in pairs:
        if pair[0] > end:
            end = pair[1]
            count += 1
    return count

## Maximal Reactangle

## Maximum swaps at most 1 swap

In [43]:
"""
Input: 2736
Output: 7236
Explanation: Swap the number 2 and the number 7.

we create hasmap as {number : index}
for each number 
    for numbers from  9 to current number:
        we check if greater number is present at index greater than current index
            if present we swap and return


edge case if there are duplicates, those will be ignored as we store max index for number in dictionary

"""

def maximumSwap(self, num):
    A = map(int, str(num))
    last = {x: i for i, x in enumerate(A)}
    for i, x in enumerate(A):
        for d in xrange(9, x, -1):
            if last.get(d, None) > i:
                A[i], A[last[d]] = A[last[d]], A[i]
                return int("".join(map(str, A)))
    return num

## Create Maximum Number

## Maximum product subarray

In [44]:
"""
cases to consider 
    1. negative numbers in array
"""

def maxProduct(self, nums):
    maxim = big = small = nums[0]
    for n in nums[1:]:
        l_big = max(n, n*big, n*small)
        l_small = min(n, n*big, n*small)
        big = l_big
        small = l_small
        maxim = max(maxim,big)
    return maxim

## Subarray Product Less Than K

In [45]:
"""
the idea is to have to pointer and update
if the product goes beyond k, 
    remove elements from left till the product goes below k
"""

def numSubarrayProductLessThanK(nums, k):
    if k <= 1: return 0
    prod = 1
    ans = left = 0
    for right, val in enumerate(nums):
        prod *= val
        while prod >= k:
            prod /= nums[left]
            left += 1
        ans += right - left + 1
    return ans

## Create Maximum Number

    Given two arrays of length m and n with digits 0-9 representing two numbers. Create the maximum number of length k <= m + n from digits of the two. The relative order of the digits from the same array must be preserved. Return an array of the k digits.

    Note: You should try to optimize your time and space complexity.


    Input:
    nums1 = [6, 7]
    nums2 = [6, 0, 4]
    k = 5
    Output:
    [6, 7, 6, 0, 4]

In [46]:
"""
The idea is to select numbers to be used from arrays.

Here we can choose 0,k elements from one array and remaining from other.

So, we find all such k combination and return the maximum numbers in order.

"""

class Solution(object):
    def maxNumber(self, nums1, nums2, k):

        def prep(nums, k):
            drop = len(nums) - k
            out = []
            for num in nums:
                while drop and out and out[-1] < num:
                    out.pop()
                    drop -= 1
                out.append(num)
            return out[:k]

        def merge(a, b):
            val = [max(a, b).pop(0) for _ in a+b]
            return val

        return max(merge(prep(nums1, i), prep(nums2, k-i))
                   for i in range(k+1)
                   if i <= len(nums1) and k-i <= len(nums2))

## Max frequency stack

    Implement FreqStack, a class which simulates the operation of a stack-like data structure.

    FreqStack has two functions:

    push(int x), which pushes an integer x onto the stack.
    pop(), which removes and returns the most frequent element in the stack.
    If there is a tie for most frequent element, the element closest to the top of the stack is removed and returned.

In [47]:
"""
maintain a counter which has freq for each element 

maintain a dictionary which has elemnts with that freqency.

"""


class FreqStack(object):

    def __init__(self):
        self.freq = collections.Counter()
        self.dict = collections.defaultdict(list)
        self.maxfreq = 0
        

    def push(self, x):
        self.freq[x] += 1
        self.dict[self.freq[x]].append(x)
        if self.freq[x] > self.maxfreq:
            self.maxfreq = self.freq[x]
        

    def pop(self):
        val = self.dict[self.maxfreq].pop()
        self.freq[val] -= 1
        if self.maxfreq and not self.dict[self.maxfreq]:
            self.maxfreq -= 1
        return val

## Maximum Product of Word Lengths

In [48]:
"""
Here the idea is to mask each word to a encoding 
i.e. binaray value for each word and combine those

if the and operation for two words returns something that means we have overlapp

"""


class Solution(object):
    def maxProduct(self, words):
        d = {}
        for word in words:
            mask = 0
            for c in set(word):
                mask = mask | (1 << (ord(c) - 97))
                
            d[mask] = max(d.get(mask, 0), len(word))
        return max([d[i] * d[j] for i in d.keys() for j in d.keys() if not i & j] + [0])

In [49]:
# zip(*grid)
# max(a, b).pop(0) for _ in a+b

In [50]:
m = [[1,2],[2,3]]

In [51]:
[c for row in m for c in row]

[1, 2, 2, 3]

## Maximum Length of Repeated Subarray

In [52]:
"""
O(n**2) dp solution
"""
def findLength(self, A, B):
    a_len = len(A)
    b_len = len(B)
    if not a_len or not b_len:
        return 0

    ret = [[0 for _ in range(a_len + 1)] for _ in range(b_len+1)]

    # A - > col
    # B -> row
    result = float('-inf')
    for row in range(1,b_len+1):
        for col in range(1, a_len+1):
            if B[row-1] == A[col-1]:
                ret[row][col] = ret[row-1][col-1] + 1
            result = max(result, ret[row][col])
    return result


"""
O(n) solution
"""

def findLength(self, A, B):
    A, res, sub = "X%sX" % "X".join(map(str, A)), 0, "X"
    # print A
    for num in B:
        sub += str(num) + "X"
        if sub in A:
            res += 1
        else: 
            sub = sub[sub[1:].index("X") + 1:]
        # print sub
    return res

## maximum width of binary tree

In [53]:
"""
The important point is incrementing position along with the depth
        traverse(root.left, depth+1, pos*2)
        traverse(root.right, depth+1, pos*2+1)
"""

def widthOfBinaryTree(root):
    left = {}
    ans = [0]
    def traverse(root, depth=0, pos=0):
        if root is None: return
        left.setdefault(depth, pos)
        ans[0] = max(ans[0], pos-left[depth]+1)
        traverse(root.left, depth+1, pos*2)
        traverse(root.right, depth+1, pos*2+1)
    traverse(root)
    return ans[0]

## Maximum Size Subarray Sum Equals k

In [54]:
def maxSubArrayLen(self, nums, k):
    d = {0:-1}
    count = 0
    ans = 0
    for i in range(len(nums)):
        count += nums[i]
        if count not in d:
            d[count] = i
        if count - k in d:
            ans = max(ans, i - d[count-k])
    return ans

## Minimum Size Subarray Sum

## Word Break 
    check if string can be formed from words

In [55]:
"""
Time complexity : O(n^2)

Space complexity : O(n). Length of dp array is n+1.

"""

def wordBreak(self, s, wordDict):
    if not wordDict or not s:
        return False
    l = len(s)
    wordDict = set(wordDict)
    dp = [False for i in range(l+1)]
    dp[0] = True

    for i in range(1, l+1):
        for j in range(i):
            if dp[j] and s[j:i] in wordDict:
                dp[i] = True
    return dp[-1]

"""
solution with memo
"""



"""
solution with dfs
"""

'\nsolution with dfs\n'

## Word Break 2

In [56]:
"""
Solution with memo
"""

def wordBreak(self, s, wordDict):
    memo = {len(s): ['']}
    def sentences(i):
        if i not in memo:
            memo[i] = [s[i:j] + (tail and ' ' + tail)
                       for j in range(i+1, len(s)+1)
                       if s[i:j] in wordDict
                       for tail in sentences(j)]
        return memo[i]
    return sentences(0)

"""
solution with dp
"""

'\nsolution with dp\n'

## Concatenated words

In [57]:
def findAllConcatenatedWordsInADict(self, words):
    d = set(words)

    def dfs(word):
        for i in range(1, len(word)):
            prefix = word[:i]
            suffix = word[i:]

            if prefix in d and suffix in d:
                return True
            if prefix in d and dfs(suffix):
                return True
            if suffix in d and dfs(prefix):
                return True

        return False

    res = []
    for word in words:
        if dfs(word):
            res.append(word)

    return res

## pattern matching

In [58]:
def z_algo(arr):
    z = [0 for i in range(len(arr))]
    left , right = 0 , 0
    for k in range(1 , len(arr)):
        if k > right:
            left = k
            right = k
            while right < len(arr) and arr[right] == arr[right-left]:
                right += 1
            z[k] = right - left
            right -= 1
        else:
            k1 = k - left
            if z[k1] < right - k + 1:
                z[k] = z[k1]
            else:
                left = k
                while right < len(arr) and arr[right] == arr[right-left]:
                    right += 1
                z[k] = right - left
                right -= 1
    return z
                
            
def makepattern(string , pattern):
    n , m = len(string) , len(pattern)
    str_arr = []
    for i in range(m):
        str_arr.append(pattern[i])
    str_arr.append('$')
    for i in range(n):
        str_arr.append(string[i])
        
    z_values = z_algo(str_arr)
    result = []
    for i in range(len(z_values)):
        if z_values[i] == m:
            result.append(i - m - 1)
    print result



if __name__ == '__main__':
    string = 'abcdeabcd'
    pattern = 'abc'
    makepattern(string , pattern)

[0, 5]


## Word Search 

In [59]:
def exist(self, board, word):
    if not board:
        return False
    for i in range(len(board)):
        for j in range(len(board[0])):
            if self.dfs(board, i, j, word):
                return True
    return False
   
def dfs(self, board, i, j, word):
    if len(word) == 0:
        return True
    if i<0 or i>=len(board) or j<0 or j>=len(board[0]) or word[0]!=board[i][j]:
        return False
    tmp = board[i][j]
    board[i][j] = "#"
    res = self.dfs(board, i+1, j, word[1:]) or self.dfs(board, i-1, j, word[1:]) \
    or self.dfs(board, i, j+1, word[1:]) or self.dfs(board, i, j-1, word[1:])
    board[i][j] = tmp
    return res

## Expressive words

In [60]:
"""
Here we use run length encoding 
we convert each word to encoding such that it generate list of 2 tuples one has characters and other has count. 


"""

def expressiveWords(self, S, words):
    def RLE(S):
        return zip(*[(k, len(list(grp)))
                     for k, grp in itertools.groupby(S)])

    R, count = RLE(S)
    ans = 0
    for word in words:
        R2, count2 = RLE(word)
        
        """
        Here we are checking if the character tuples are same or not 
        """
        if R2 != R: continue
        ans += all(c1 >= max(c2, 3) or c1 == c2
                   for c1, c2 in zip(count, count2))

    return ans

## Valid Word Abbreviation

In [61]:
"""
word = word
abbr = ["word", "1ord", "w1rd", "wo1d", "wor1", "2rd", "w2d", "wo2", "1o1d", "1or1", "w1r1", "1o2", "2r1", "3d", "w3", "4"]
"""

def validWordAbbreviation(word, abbr):
    left, right = 0,0
    stack = []
    while left < len(abbr) and right < len(word):
        if abbr[left] == word[right]:
            left += 1
            right += 1
        elif not abbr[left].isalpha():
            k = left
            l = []
            if abbr[left] == "0":
                return False
            while k < len(abbr) and not abbr[k].isalpha():
                l.append(abbr[k])
                k += 1
            num = int("".join(l))
            left += k - left
            right += num
        else:
            return False
    return left == len(abbr) and right == len(word)

## map * vs zip *

In [62]:
words = ["abcd", "xyz"]

In [63]:
map(None, *words)

[('a', 'x'), ('b', 'y'), ('c', 'z'), ('d', None)]

In [64]:
zip(*words)

[('a', 'x'), ('b', 'y'), ('c', 'z')]

## Trie

In [65]:
import collections 

Trie = lambda: collections.defaultdict(Trie)
trie = Trie()

## Trie code

In [66]:
words = ["abcd", "bcde", "def", "abc"]


Trie = lambda: collections.defaultdict(Trie)
trie = Trie()
END = True

for i, word in enumerate(words):
    reduce(dict.__getitem__, word, trie)[END] = i

trie

defaultdict(<function __main__.<lambda>>,
            {'a': defaultdict(<function __main__.<lambda>>,
                         {'b': defaultdict(<function __main__.<lambda>>,
                                      {'c': defaultdict(<function __main__.<lambda>>,
                                                   {True: 3,
                                                    'd': defaultdict(<function __main__.<lambda>>,
                                                                {True: 0})})})}),
             'b': defaultdict(<function __main__.<lambda>>,
                         {'c': defaultdict(<function __main__.<lambda>>,
                                      {'d': defaultdict(<function __main__.<lambda>>,
                                                   {'e': defaultdict(<function __main__.<lambda>>,
                                                                {True: 1})})})}),
             'd': defaultdict(<function __main__.<lambda>>,
                         {'e': defaul

In [67]:
trie.values()

[defaultdict(<function __main__.<lambda>>,
             {'b': defaultdict(<function __main__.<lambda>>,
                          {'c': defaultdict(<function __main__.<lambda>>,
                                       {True: 3,
                                        'd': defaultdict(<function __main__.<lambda>>,
                                                    {True: 0})})})}),
 defaultdict(<function __main__.<lambda>>,
             {'c': defaultdict(<function __main__.<lambda>>,
                          {'d': defaultdict(<function __main__.<lambda>>,
                                       {'e': defaultdict(<function __main__.<lambda>>,
                                                    {True: 1})})})}),
 defaultdict(<function __main__.<lambda>>,
             {'e': defaultdict(<function __main__.<lambda>>,
                          {'f': defaultdict(<function __main__.<lambda>>,
                                       {True: 2})})})]

## suffix Tree

## prefix tree

## Longest word in a list

In [68]:
import collections
"""
["a", "banana", "app", "appl", "ap", "apply", "apple"]
Output: "apple"
"""

def longestWord(words):
    wordset = set(words)
    words.sort(key = lambda c: (-len(c), c))
    print(words)
    for word in words:
        if all(word[:k] in wordset for k in xrange(1, len(word))):
            return word

    return ""

"""Solution based on trie"""

def longestWord(words):
    Trie = lambda: collections.defaultdict(Trie)
    trie = Trie()
    END = True

    for i, word in enumerate(words):
        reduce(dict.__getitem__, word, trie)[END] = i

    stack = trie.values()
    ans = ""
    while stack:
        cur = stack.pop()
        if END in cur:
            word = words[cur[END]]
            if len(word) > len(ans) or len(word) == len(ans) and word < ans:
                ans = word
            stack.extend([cur[letter] for letter in cur if letter != END])

    return ans

In [69]:
words = ["a", "banana", "app", "appl", "ap", "apply", "apple"]
longestWord(words)

'apple'

Sort words by length then lexographically
    
    words.sort(key = lambda c: (-len(c), c))

## WordSquares

    First we create a prefix dictionary which has start word as key and words starting with that key in a list.
    The idea is to select every word as a start word and then try out candidates for 2nd word. 
    
    ["area","lead","wall","lady","ball"]
    eg. we selected word "wall" as starting word now in next loop we will zip the array so that we will get seprate candidate to start next word with.
    for 2nd we will select all words starting with 'a' as length of word array is 1.
        wall
        area
    for 3rd we will select words starting with 'le'
        wall
        area
        lead
    for last we will select words starting with 'lad'
        wall
        area
        lead
        lady

In [70]:
def wordSquares(words):
    n = len(words[0])
    fulls = collections.defaultdict(list)
    for word in words:
        for i in range(n):
            fulls[word[:i]].append(word)
    def build(square):
        if len(square) == n:
            squares.append(square)
            return
        for word in fulls[''.join(zip(*square)[len(square)])]:
            build(square + [word])
    squares = []
    for word in words:
        build([word])
    return squares

## regex example
    remove punctuations

In [71]:
import re
re.sub('[!?\',;.]',"", "ab?.'!ab")

'abab'

         we may use length of string as a key to dictionary to store words

## Shortest Path Visiting All Nodes

In [72]:
"""BFS Solution"""

"""
Here memo is not required it just improves the computation time ignoring revisits

The idea is that as the numbers are marked from 0 to n-1 we can assign weight to each node as power of two 
so out final answer will reach when sum of all node visits is 2 ** n -1 

we use bfs for this and for every step we calculate cost of moving to its neighbours when we reach the final sum
we can return num steps for the first occurance of sum == final

"""

def shortestPathLength(graph):
    memo, final, q, steps = set(), (1 << len(graph)) - 1, [(i, 1 << i) for i in range(len(graph))], 0
    while True:
        new = set()
        for node, state in q:
            if state == final: return steps
            for v in graph[node]:
                if (state | 1 << v, v) not in memo:
                    new.add((v, state | 1 << v))
                    memo.add((state | 1 << v, v))
        q = new
        steps += 1

## Number of distinct subsequences
    dp[i][j] = dp[i-1][j] + dp[i-1][j-1]*(s[i-1] == t[j-1])

In [73]:
"""
Input: S = "rabbbit", T = "rabbit"
Output: 3

dp[i][j] = dp[i-1][j] + dp[i-1][j-1]*(s[i-1] == t[j-1])
"""

def numDistinct1(s, t):
    l1, l2 = len(s)+1, len(t)+1
    dp = [[1] * l2 for _ in xrange(l1)]
    for j in xrange(1, l2):
        dp[0][j] = 0
    for i in xrange(1, l1):
        for j in xrange(1, l2):
            dp[i][j] = dp[i-1][j] + dp[i-1][j-1]*(s[i-1] == t[j-1])
    return dp[-1][-1]

numDistinct1("rabbbbit", "rabbbit")

4

## All Nodes Distance K in Binary Tree

In [74]:
"""
In this solution we annotate every node with its parent node and apply bfs to the generated graph
"""

def distanceK(self, root, target, K):
    def dfs(root, par=None):
        if root:
            root.par = par
            dfs(root.left, root)
            dfs(root.right, root)
    dfs(root)

    seen = set()
    q = [(target, 0)]
    result = []
    while q:
        node, depth = q.pop(0)
        seen.add(node)
        if depth == K:
            result.append(node.val)
            continue
        for nei in (node.par, node.left, node.right):
            if nei and nei not in seen:
                q.append((nei, depth+1))
    return result

## Longest increasing subsequence

In [75]:
"""
Normal O(n**2) solution
"""

def lengthOfLIS(self, nums):
    if not nums:
        return 0
    length = len(nums)
    ret = [1] * length
    for i in range(1, length):
        for j in range(i):
            if nums[i] > nums[j]:
                ret[i] = max(ret[i], ret[j] + 1)
    return max(ret)


"""
Improved solution with bisect

The idea is to generate array with increasing numbers if the number is greater we append 
if we find any number which is less than a number we replace that number so that we keep only smaller numbers in array 
so if any new number comes we can add it to array

"""

import bisect

def lengthOfLIS(nums):
    ret = [float('inf')] * len(nums)
    result = 0
    for num in nums:
        index = bisect.bisect_left(ret, num)
        ret[index] = num
        if result < index + 1:
            result = index + 1
    return result

## Longest Consecutive Sequence

In [76]:
"""
Input: [100, 4, 200, 1, 3, 2]
Output: 4
"""

def longestConsecutive(nums):

    longest = 0
    n = set(nums)

    for i in nums:
        if (i - 1) not in n: 
            nxt = i + 1
            current = 1
            while nxt in n:
                nxt += 1
                current += 1
            longest = max(longest, current)

    return longest

## Target Sum

In [77]:
"""
Input: nums is [1, 1, 1, 1, 1], S is 3. 
Output: 5
Explanation: 

-1+1+1+1+1 = 3
+1-1+1+1+1 = 3
+1+1-1+1+1 = 3
+1+1+1-1+1 = 3
+1+1+1+1-1 = 3

There are 5 ways to assign symbols to make the sum of nums be target 3.

"""

def findTargetSumWays(nums, S):
    if not nums:
        return 0
    stack = {0:1}
    for num in nums:
        new_stack = {}
        for element in stack:
            new_stack[element+num] = new_stack.get(element+num, 0) + stack[element]
            new_stack[element-num] = new_stack.get(element-num, 0) + stack[element]
        stack = new_stack
    return stack.get(S, 0)

## Shortest Bridge

In [78]:
import collections
def shortestBridge(A):
    row_len = len(A)
    col_len = len(A[0])
    queue = collections.deque()
    visited = set()

    def dfs(row, col):
        if 0 <= row < row_len and 0 <= col < col_len and A[row][col] == 1 and (row, col) not in visited:
            visited.add((row, col))
            queue.append((row, col, 0))
            dfs(row+1, col)
            dfs(row-1, col)
            dfs(row, col-1)
            dfs(row, col+1)

    b = False
    for row in range(row_len):
        for col in range(col_len):
            if A[row][col] == 1:
                dfs(row, col)
                b = True
                break
        if b: break
    
    while queue:
        row, col, d = queue.popleft()
        if A[row][col] == 1 and (row, col) not in visited:
            return d
        visited.add((row, col))
        for dx, dy in [(1,0), (-1,0), (0,1), (0,-1)]:
            new_row = row + dx
            new_col = col + dy
            if 0 <= new_row < row_len and 0 <= new_col < col_len and (new_row, new_col) not in visited:
                queue.append((new_row, new_col, d+1))

## min depth of binary search tree

In [79]:
def minDepth(self, root):

    if root is None:
        return 0
    stack = [root]
    new_stack = []
    depth = 0
    while True:
        new_stack = []
        while stack:
            node = stack.pop()
            if node.left is None and node.right is None:
                return depth + 1
            if node.left is not None:
                new_stack.append(node.left)
            if node.right is not None:
                new_stack.append(node.right)
        stack.extend(new_stack)
        depth += 1
        
def minDepth(self, root):
        """
        :type root: TreeNode
        :rtype: int
        """
        def mind(root):
            if root is None:
                return 0
            elif root.left is None and root.right is None:
                return 1
            elif root.left is None:
                return 1 + mind(root.right)
            elif root.right is None:
                return 1 + mind(root.left)
            else:
                return 1 + min(mind(root.left), mind(root.right))
        return mind(root)

## min falling path sum

In [80]:
def minFallingPathSum(self, A):
    """
    :type A: List[List[int]]
    :rtype: int
    """
    while len(A) >= 2:
        row = A.pop()
        for i in range(len(row)):
            A[-1][i] += min(row[max(i-1,0): min(i+2, len(row))])
    return min(A[-1])

## num subarray with sum

In [81]:
"""
Input: A = [1,0,1,0,1], S = 2
Output: 4
Explanation: 
The 4 subarrays are bolded below:
[1,0,1,0,1]
[1,0,1,0,1]
[1,0,1,0,1]
[1,0,1,0,1]
"""


def numSubarraysWithSum(self, A, S):
    P = [0]
    for x in A: P.append(P[-1] + x)
    count = collections.Counter()

    ans = 0
    for x in P:
        ans += count[x]
        count[x + S] += 1

    return ans

## number of LIS

In [82]:
def findNumberOfLIS(nums):
    N = len(nums)
    if N <= 1: return N
    lengths = [0] * N
    counts = [1] * N

    for i, num in enumerate(nums):
        for j in xrange(i):
            if nums[j] < nums[i]:
                if lengths[j] >= lengths[i]:
                    lengths[i] = 1 + lengths[j]
                    counts[i] = counts[j]
                elif lengths[j] + 1 == lengths[i]:
                    counts[i] += counts[j]

    longest = max(lengths)
    return sum(c for i, c in enumerate(counts) if lengths[i] == longest)

## Longest Mountain in array

In [83]:
def longestMountain(self, A):
    length = len(A)
    start = 0
    ans = 0

    while start < length:
        test = start
        if test < length-1 and A[test] < A[test+1]:
            while test < length-1 and A[test] < A[test+1]:
                test += 1
            if test < length-1 and A[test] > A[test+1]:
                while test < length-1 and A[test] > A[test+1]:
                    test += 1
                ans = max(ans, test - start + 1)
        start = max(test, start+1)
    return ans

## Longest increasing sequence in a matrix

In [84]:
"""
DFS solution apply dfs at every point and store result in a memo to reuse.
"""

from collections import defaultdict
class Solution(object):
    def longestIncreasingPath(self, matrix):
        """
        :type matrix: List[List[int]]
        :rtype: int
        """
        dct = defaultdict(int)
        
        '''
        Compute DFS from given cell
        '''
        def dfs(matrix, i, j):
            if (i, j) in dct:
                return dct[(i, j)]
            if not (0<=i<len(matrix) and 0<=j<len(matrix[0])):
                return 0
            res = [1] # If no neighbors exist, return 1
            for row, col in [(i-1, j), (i+1, j),(i, j-1), (i, j+1)]:
                if (0<=row<len(matrix) and 0<=col<len(matrix[0])):
                    if matrix[row][col]>matrix[i][j]:
                        res.append(1+dfs(matrix, row, col))
            dct[(i, j)] = max(res)
            return dct[(i, j)]
        
        '''
        Run DFS from every cell
        '''
        currmax = 0
        for i in range(0, len(matrix)):
            for j in range(0, len(matrix[i])):
                currmax = max(dfs(matrix, i, j), currmax)
        return currmax

## minimum window substring

In [85]:
"""
Input: 
S = "abcdebdde", T = "bde"
Output: "bcde"
"""
def minWindow(self, S, T):
    col_len = len(S)
    row_len = len(T)

    dp = [[(float('inf')) for _ in range(col_len + 1)] for _ in range(row_len+1)]
    for row in range(1, row_len+1):
        for col in range(1, col_len+1):
            if S[col-1] == T[row-1]:
                if row == 1:
                    dp[row][col] = 1
                else:
                    dp[row][col] = dp[row-1][col-1] + 1
            else:
                dp[row][col] = dp[row][col-1] + 1

    min_val = float('inf')
    index = 0
    for i in range(0, col_len+1):
        if dp[-1][i] < min_val:
            min_val = dp[-1][i]
            index = i
    return S[index-min_val: index] if min_val != float('inf') else ""

## Longest Repeating Character Replacement

In [86]:
"""
Input:
s = "AABABBA", k = 1

Output:
4

"""

def characterReplacement(self, s, k):
    result = 0
    window = collections.defaultdict(int)
    i = 0
    for j in range(len(s)):
        window[s[j]] += 1
        while sum(window.values()) - max(window.values()) > k:
            if window[s[i]] == 1:
                del window[s[i]]
            else:
                window[s[i]] -= 1
            i += 1
        result = max(result, j - i + 1)
    return result

## Longest substring with at least k repeating characters

In [87]:
def longestSubstring(self, s, k):
    for c in set(s):
        if s.count(c) < k:
            return max(self.longestSubstring(t, k) for t in s.split(c))
    return len(s)

## isSubsequence

In [88]:
def isSubsequence(s, t):
    j = 0
    for i in range(len(s)):
        while len(t) > j and s[i] == t[j]:
            j += 1
    return j == len(t)

In [89]:
"""
This is faster approach for is subsequence
"""

s = "dsapxple"
word = "apple"
def isSubsequence(s, t):
    it = iter(s)
    return all(word in it for word in t)

In [90]:
isSubsequence(s, word)

True

## Longest Substring with At Most Two Distinct Characters

In [91]:
"""
Input: "eceba"
Output: 3
Explanation: t is "ece" which its length is 3.
"""

def lengthOfLongestSubstringTwoDistinct(self, s):
    c = collections.Counter()
    result = 0
    j = 0
    for i in range(len(s)):
        c[s[i]] += 1
        while j < len(s) and len(c) > 2:
            if c[s[j]] == 1:
                del c[s[j]]
            else:
                c[s[j]] -= 1
            j += 1
        result = max(result, i - j + 1)
    return result

In [92]:
"""
longest paliandrome o(n) solution
"""
def longestPalindrome(s):
    if not s:
        return ""

    maxm = 1
    n = len(s)
    start = 0
    for i in range(n):
        if i - maxm >= 1 and s[i - maxm - 1:i + 1] == s[i - maxm - 1:i + 1][::-1]:
            start = i - maxm - 1
            maxm += 2
            continue
        if i - maxm >= 0 and s[i - maxm:i + 1] == s[i - maxm:i + 1][::-1]:
            start = i - maxm
            maxm += 1
    return s[start:start + maxm]

    sorted to string returns list
        sorted("ABCD")
        ['A', 'B', 'C', 'D']

    check even odd
       n & 1 == 0

## insert interval / merge Interval
    array must be sorted according to start time

In [93]:
"""
Insert interval


here we keep 3 different arrays one for start part, one for interval part we merge any overlapping intervals with that

third is non overalpping part after interval
"""

def insert(intervals, newInterval):
    start = []
    index = 0
    for index, i in enumerate(intervals):
        if i.start > newInterval.end:
            index -= 1
            break
        elif i.end < newInterval.start:
            start += i,
        else:
            newInterval.start, newInterval.end = min(newInterval.start, i.start), max(newInterval.end, i.end)
    return start + [newInterval] + intervals[index+1:]

## isIsomorphic
    len(set(zip(s, t))) == len(set(s)) == len(set(t))

### Reverse a linked list

In [94]:
def reverseLL(head):    
    def reverse(node, prev=None):
        if not node:
                return prev
        n = node.next
        node.next = prev
        return reverse(n, node)
    return reverse(head)


def reverseLL(head):
    prev = None
    while head:
        node = head.next
        head.next = prev
        prev = head
        head = node
    return prev

## Word Search 2
    words = ["oath","pea","eat","rain"] and board =
    [
      ['o','a','a','n'],
      ['e','t','a','e'],
      ['i','h','k','r'],
      ['i','f','l','v']
    ]

    Output: ["eat","oath"]

In [95]:
"""
The task is to find whether the words in dictionary are present in the matrix or not
For that we will create a trie data structure which will help us traversing through the loop.
and we perform dfs sending trie values
"""

class Solution:
    def findWords(self, board, words):
        trie={}
        for w in words:
            t=trie
            for c in w:
                if c not in t:
                    t[c]={}
                t=t[c]
            t['#']='#'
        self.res=set()
        self.used=[[False]*len(board[0]) for _ in range(len(board))]
        for i in range(len(board)):
            for j in range(len(board[0])):
                self.find(board,i,j,trie,'')
        return list(self.res)
    
    def find(self,board,i,j,trie,pre):
        if '#' in trie:
            self.res.add(pre)
        if i<0 or i>=len(board) or j<0 or j>=len(board[0]):
            return
        if not self.used[i][j] and board[i][j] in trie:
            self.used[i][j]=True
            self.find(board,i+1,j,trie[board[i][j]],pre+board[i][j])
            self.find(board,i,j+1,trie[board[i][j]],pre+board[i][j])
            self.find(board,i-1,j,trie[board[i][j]],pre+board[i][j])
            self.find(board,i,j-1,trie[board[i][j]],pre+board[i][j])
            self.used[i][j]=False

## Word Ladder 2

In [96]:
"""

beginWord = "hit",
endWord = "cog",
wordList = ["hot","dot","dog","lot","log","cog"]

Output:
[
  ["hit","hot","dot","dog","cog"],
  ["hit","hot","lot","log","cog"]
]

we create a dictionary that has key which is end word in current iteration
we process each iteration and store results into an array which is used in next iteration
we keep on removing the elements from wordList as we proceed.

"""
def findLadders(self, beginWord, endWord, wordList):
    wordList = set(wordList)
    res = []
    layer = {}
    layer[beginWord] = [[beginWord]]
    while layer:
        newLayer = collections.defaultdict(list)
        for w in layer:
            if w == endWord:
                res.extend(l for l in layer[w])
            for i in range(len(w)):
                for c in 'abcdefghijklmnopqrstuvwxyz':
                    newWord = w[:i] + c + w[i+1:]
                    if newWord in wordList:
                        newLayer[newWord] += [l + [newWord] for l in layer[w]]
        wordList -= set(newLayer.keys())
        layer = newLayer
    return res

## word ladder

In [97]:
class Solution(object):
    def ladderLength(self, beginWord, endWord, wordList):
        wordList = set(wordList)
        queue = collections.deque([[beginWord, 1]])
        while queue:
            word, length = queue.popleft()
            if word == endWord:
                return length
            for i in range(len(word)):
                for c in 'abcdefghijklmnopqrstuvwxyz':
                    next_word = word[:i] + c + word[i+1:]
                    if next_word in wordList:
                        wordList.remove(next_word)
                        queue.append([next_word, length + 1])
        return 0

## Surrounded Regions

In [98]:
class Solution:
    def solve(self, board):
        """
        start with the edge elements row-wise and column-wise
        """
        if len(board) == 0:
            return
        row_num = len(board)
        col_num = len(board[0])
        
        def dfs(i,j):
            if 0 <= i < row_num and 0 <= j < col_num and board[i][j] == 'O':
                board[i][j] = 'S'
                for x, y in [(-1,0), (1,0), (0,-1), (0,1)]:
                    dfs(i+x, j+y)

        for i in range(row_num):
            if board[i][0] == 'O':
                dfs(i, 0)
            if board[i][col_num-1] == 'O':
                dfs(i, col_num-1)
        for i in range(col_num):
            if board[0][i] == 'O':
                dfs(0, i)
            if board[row_num-1][i] == 'O':
                dfs(row_num-1, i)
        for row in range(row_num):
            for col in range(col_num):
                if board[row][col] == 'S':
                    board[row][col] = 'O'
                    continue
                board[row][col] = 'X'

## Clone graph

In [99]:
def cloneGraph(self, node):
    if not node:
        return 
    nodeCopy = UndirectedGraphNode(node.label)
    dic = {node: nodeCopy}
    queue = collections.deque([node])
    while queue:
        node = queue.popleft()
        for neighbor in node.neighbors:
            if neighbor not in dic: # neighbor is not visited
                neighborCopy = UndirectedGraphNode(neighbor.label)
                dic[neighbor] = neighborCopy
                dic[node].neighbors.append(neighborCopy)
                queue.append(neighbor)
            else:
                dic[node].neighbors.append(dic[neighbor])
    return nodeCopy

## Distinct Subsequences

In [100]:
# O(m*n) space 
def numDistinct1(self, s, t):
    l1, l2 = len(s)+1, len(t)+1
    dp = [[1] * l2 for _ in xrange(l1)]
    for j in xrange(1, l2):
        dp[0][j] = 0
    for i in xrange(1, l1):
        for j in xrange(1, l2):
            dp[i][j] = dp[i-1][j] + dp[i-1][j-1]*(s[i-1] == t[j-1])
    return dp[-1][-1]
  
# O(n) space  
def numDistinct(self, s, t):
    l1, l2 = len(s)+1, len(t)+1
    cur = [0] * l2
    cur[0] = 1
    for i in xrange(1, l1):
        pre = cur[:]
        for j in xrange(1, l2):
            cur[j] = pre[j] + pre[j-1]*(s[i-1] == t[j-1])
    return cur[-1]

## meeting rooms 2

In [101]:
class Solution(object):
    def minMeetingRooms(self, intervals):

        if not intervals:
            return 0
        d = {}
        for i in intervals:
            d[i.start] = d.get(i.start, 0) + 1
            d[i.end] = d.get(i.end, 0) - 1
        max_value = 1 
        value = 0
        for key in sorted(d.keys()):
            value += d[key]
            max_value = max(max_value, value)
        return max_value

## median data stream

In [102]:
class MedianFinder:
    def __init__(self):
        self.small = []
        self.large = []

    def addNum(self, num):
        if len(self.small) == len(self.large):
            heappush(self.large, -heappushpop(self.small, -num))
        else:
            heappush(self.small, -heappushpop(self.large, num))

    def findMedian(self):
        if len(self.small) == len(self.large):
            return float(self.large[0] - self.small[0]) / 2.0
        else:
            return float(self.large[0])
        
        
"""using bisect"""

class MedianFinder:
    def __init__(self):
        self.array = []
        self.count = 0

    def addNum(self, num):
        bisect.insort(self.array, num)
        self.count += 1

    def findMedian(self):
        if self.count == 0:
            return 0
        elif self.count % 2 != 0:
            return self.array[self.count//2] * 1.0
        else:
            return (self.array[self.count//2] + self.array[self.count//2 -1]) / 2.0

## serailize deserialize binary tree

In [103]:
class Codec:
    lst = []

    def preorder(self, root):
        if root:
            self.lst.append(str(root.val))
            self.preorder(root.left)
            self.preorder(root.right)
        else:
            self.lst.append("#")
        
    def serialize(self, root):
        """Encodes a tree to a single string.
        
        :type root: TreeNode
        :rtype: str
        """
        self.lst = [] 
        self.preorder(root)
        return " ".join(self.lst)
        
        

    def deserialize(self, data):
        """Decodes your encoded data to tree.
        
        :type data: str
        :rtype: TreeNode
        """
        def func():
            value = next(var)
            if value == "#":
                return None
            node = TreeNode(int(value))
            node.left = func()
            node.right = func()
            return node
            
        var = iter(data.split())
        return func()

    There's an alternative to the StopIteration by using next(iterator, default_value).

    For exapmle:

    >>> a = iter('hi')
    >>> print next(a, None)
    h
    >>> print next(a, None)
    i
    >>> print next(a, None)
    None

## smallest range

    find smallest range for 3 sorted array's
   

In [104]:
"""
The idea is to get min from each array and find ranges at each step
 
"""

def smallestRange(self, A):
    pq = [(row[0], i, 0) for i, row in enumerate(A)]
    heapq.heapify(pq)
    
    ans = -1e9, 1e9
    right = max(row[0] for row in A)
    while pq:
        left, i, j = heapq.heappop(pq)
        if right - left < ans[1] - ans[0]:
            ans = left, right
        if j + 1 == len(A[i]):
            return ans
        v = A[i][j+1]
        right = max(right, v)
        heapq.heappush(pq, (v, i, j+1))

## Range sum query 2 - D

In [105]:
class NumMatrix:

    def __init__(self, matrix):
        self.sums = [[0] * (len(matrix and matrix[0]) + 1) for _ in range(len(matrix) + 1)]
        for i in range(len(matrix)):
            for j in range(len(matrix and matrix[0])):
                self.sums[i + 1][j + 1] = self.sums[i][j + 1] + self.sums[i + 1][j] - self.sums[i][j] + matrix[i][j]

    def sumRegion(self, row1, col1, row2, col2):
        return self.sums[row2 + 1][col2 + 1] - self.sums[row2 + 1][col1] - self.sums[row1][col2 + 1] + self.sums[row1][col1]

## House Robber 3 
    using tree

In [106]:
 def rob(self, root):
    def rob_helper(root):
        if root == None:
            return (0, 0)
        x = rob_helper(root.left)
        y = rob_helper(root.right)
        return (max(root.val + x[-1] + y[-1], x[0] + y[0]), x[0] + y[0])
    return rob_helper(root)[0]

## Tic tac toe

In [107]:
class TicTacToe(object):

    def __init__(self, n):
        self.row, self.col, self.diag, self.anti_diag, self.n = [0] * n, [0] * n, 0, 0, n
        

    def move(self, row, col, player):
        offset = player * 2 - 3
        self.row[row] += offset
        self.col[col] += offset
        if row == col:
            self.diag += offset
        if row + col == self.n - 1:
            self.anti_diag += offset
        if self.n * offset in [self.row[row], self.col[col], self.diag, self.anti_diag]:
            return player
        return 0

## Counter
    & intersection of two counter 
    | union of two counter

In [134]:
from collections import Counter

In [135]:
c1, c2 = Counter([1,1,2,3,4,4]), Counter([1,1,3,4,4,4])

In [136]:
c1 & c2

Counter({1: 2, 3: 1, 4: 2})

In [137]:
c1 | c2

Counter({1: 2, 2: 1, 3: 1, 4: 3})

In [138]:
c1 + c2

Counter({1: 4, 2: 1, 3: 2, 4: 5})

## Longest increasing path in a matrix

In [112]:
"""
We apply dfs at each node and we store results of each node in dictionary
"""

def longestIncreasingPath(self, A):
    
    if not A or not A[0]:
        return 0
    row_len = len(A)
    col_len = len(A[0])
    d = {}
    result = 0
    def dfs(row, col):
        ret = 1
        if (row, col) in d:
            return d[(row,col)]
        for new_row, new_col in [(row+x, col+y) for x, y in [(1,0),(-1,0),(0,1),(0,-1)]]:
            if 0 <= new_row < row_len and 0<= new_col < col_len and A[row][col] < A[new_row][new_col]:
                ret = max(ret, 1+dfs(new_row, new_col))
        d[(row, col)] = ret
        return ret

    for row in range(row_len):
        for col in range(col_len):
            result = max(result, dfs(row, col))
    return result

## Reconstruct Itinerary

## remove duplicates and return in lexo order

In [113]:
def removeDuplicateLetters(self, s):
    for c in sorted(set(s)):
        suffix = s[s.index(c):]
        if set(suffix) == set(s):
            return c + self.removeDuplicateLetters(suffix.replace(c, ''))
    return ''

## K pair with smallest sum

In [114]:
class Solution(object):
    def kSmallestPairs(self, nums1, nums2, k, heap=[]):
        for n1 in nums1:
            for n2 in nums2:
                if len(heap) < k: heapq.heappush(heap, (-n1-n2, [n1, n2]))
                else:
                    if heap and -heap[0][0] > n1 + n2:
                        heapq.heappop(heap)
                        heapq.heappush(heap, (-n1-n2, [n1, n2]))
                    else: break
        return [heapq.heappop(heap)[1] for _ in range(k) if heap]

## Number of Unique BST

In [115]:
def numTrees(self, n):
    ## for any increasing numbers no. of binary search trees are same.
    ## 1,2,3 will have same number of bst as 5,6,7

    ## to find number of binary search trees for n
    ## iterate through the array with each array element as a root.
    ## multiply number of left sub trees with number of right subtrees

    res = [0 for i in range(n+1)]
    res[0] = 1
    res[1] = 1

    for i in range(2,n+1):
        for j in range(0,i):
            # print(i, j, i-j-1)
            res[i] += res[j] * res[i-j-1]

    return res[n]

## Generate Unique BST

In [116]:
class Solution:
    def generateTrees(self, n):
        if n == 0:
            return []
        def node(val, left, right):
            node = TreeNode(val)
            node.left = left
            node.right = right
            return node
        def trees(first, last):
            return [node(root, left, right)
                    for root in range(first, last+1)
                    for left in trees(first, root-1)
                    for right in trees(root+1, last)] or [None]
        return trees(1, n)

## Maximize the distance to closest

In [117]:
"""
Input: [1,0,0,0]
Output: 3
Explanation: 
If Alex sits in the last seat, the closest person is 3 seats away.
This is the maximum distance possible, so the answer is 3.

"""


def maxDistToClosest(self, seats):
    l,r = 0, 0
    ret = 0
    while r < len(seats):
        if l == 0 and seats[l] == 0:
            ret = max(ret, r-l)
        if r == len(seats) - 1 and seats[r] == 0:
            ret = max(ret, r-l)
        if seats[r] == 1:
            ret = max(ret, (r-l)/2)
            l = r
            r += 1
        else:
            r += 1
    return ret

## Exam Room

## has path sum root to leaf

In [118]:
def hasPathSum(self, root, target):
    def traverse(root, s=0):
        if root is None:
            return False
        val = root.val
        if root.left is None and root.right is None and s + val == target:
            return True
        return traverse(root.left, s+val) or traverse(root.right, s+val)
    return traverse(root)

## path sum 2
    return sum

In [119]:
def pathSum(self, root, target):
    result = []
    def traverse(root, s=0, ret=[]):
        if root is None:
            return
        val = root.val
        if root.left is None and root.right is None and s + val == target:
            ret += [val]
            result.append(ret)
        traverse(root.left, s+val, ret+[val])
        traverse(root.right, s+val, ret+[val])
    traverse(root)
    return result

## path sum 3 & 4

## buy sell stocks with cooldown

In [120]:
"""

The key is 3 states and 5 edges for state transition. 3 states are notHold (stock), hold (stock), and notHold_cooldown. The initial values of the latter two are negative infinity since they are meaningless, i.e. you won't hold stocks at first and there's no cooldown at first. The 5 edges:

hold -----do nothing----->hold

hold -----sell----->notHold_cooldown

notHold -----do nothing -----> notHold

notHold -----buy-----> hold

notHold_cooldown -----do nothing----->notHold

"""

def maxProfit(self, prices):
    notHold, notHold_cooldown, hold = 0, float('-inf'), float('-inf')
    for p in prices:
        hold, notHold, notHold_cooldown = max(hold, notHold - p), max(notHold, notHold_cooldown), hold + p
    return max(notHold, notHold_cooldown)

In [121]:
nums = [1,1,2,3,3,3,4]

In [122]:
sorted(nums, key=nums.count, reverse=True)

[3, 3, 3, 1, 1, 2, 4]

## Largest Rectangular Area Histogram

In [123]:
"""

    The O(n^2) solution with O(1) space throws TLE
    Try using more space.


    maxArea = 0
    num = len(heights)
    for i in range(num):
        minHeight = float('inf')
        for j in range(i, num):
            minHeight = min(minHeight, heights[j])
            maxArea = max(maxArea, (j-i+1) * minHeight)
    return maxArea
    
    
    
    divide and cocur also throws TLE when the array is sorted the complexity becomes O(n^2)
    
    def largestRectangleArea(self, heights):
        def calculateArea(heights, start, end):
            if start > end:
                return 0
            minHeight = start
            for i in range(start, end+1):
                if heights[minHeight] > heights[i]:
                    minHeight = i
            return max(heights[minHeight]*(end-start+1), calculateArea(heights, start, minHeight-1), calculateArea(heights, minHeight+1, end))
        return calculateArea(heights, 0, len(heights)-1)
    
    
    works : Using stacks O(n) time O(n) space
    
    def largestRectangleArea(self, heights):
        maxArea = 0
        stack = []
        for i, val in enumerate(heights+[0]):
            while stack and stack[-1][1] > val:
                index, height = stack.pop()
                width = i - stack[-1][0] - 1 if stack else i
                maxArea = max(maxArea, width * height)
            stack.append((i, val))
        return maxArea
    """ 
    
def largestRectangleArea(self, heights):
    maxArea = 0
    stack = []
    for i, val in enumerate(heights+[0]):
        while stack and stack[-1][1] > val:
            index, height = stack.pop()
            width = i - stack[-1][0] - 1 if stack else i
            maxArea = max(maxArea, width * height)
        stack.append((i, val))
    return maxArea

## calculate division

In [124]:
def calcEquation(self, equations, values, queries):
    quot = collections.defaultdict(dict)
    for (source, dest), val in zip(equations, values):
        quot[source][dest] = val
        quot[dest][source] = 1.0/ val
        quot[source][source] = quot[dest][dest] = 1
    for k in quot:
        for i in quot[k]:
            for j in quot[k]:
                quot[i][j] = quot[i][k] * quot[k][j]
    return [quot[source].get(dest, -1) for source, dest in queries]

## Longest Palindrome

In [125]:
def longestPalindrome(self, s):
    max_len=1
    end=1
    for i in range(1,len(s)):
        if i-max_len-1>=0 and s[i-max_len-1:i+1]==s[i-max_len-1:i+1][::-1]:
            max_len+=2
            end=i+1
            continue
        if s[i-max_len:i+1]==s[i-max_len:i+1][::-1]:
            max_len+=1
            end=i+1
    return s[end-max_len:end]

## MAximal reactangle

In [126]:
def maximalRectangle(self, matrix):
    if not matrix or not matrix[0]:
        return 0
    n = len(matrix[0])
    height = [0] * (n + 1)
    ans = 0
    for row in matrix:
        for i in xrange(n):
            height[i] = height[i] + 1 if row[i] == '1' else 0
        stack = [-1]
        for i in xrange(n + 1):
            while height[i] < height[stack[-1]]:
                h = height[stack.pop()]
                w = i - 1 - stack[-1]
                ans = max(ans, h * w)
            stack.append(i)
    return ans

In [127]:
l = [
  "abcd",
  "bnrt",
  "crmy",
  "dtye"
]

In [131]:
map(None,*l)

[('a', 'b', 'c', 'd'),
 ('b', 'n', 'r', 't'),
 ('c', 'r', 'm', 'y'),
 ('d', 't', 'y', 'e')]

In [133]:
def letterCombinations(self, digits):
        if len(digits) == 0:
            return []
        mapping = {'2': 'abc', '3': 'def', '4': 'ghi', '5': 'jkl', 
                   '6': 'mno', '7': 'pqrs', '8': 'tuv', '9': 'wxyz'}
        if len(digits) == 1:
            return list(mapping[digits[0]])
        result = []
        for digit in digits:
            temp = []
            for ch in mapping[digit]:
                if not result:
                    temp.append(ch)
                else:
                    temp.extend([i+ch for i in result])
            result = temp
        return result

    less than operation with set

    sorted(string)  returns a list

## reverse k groups

In [140]:
def reverseKGroup(self, head, k):
    dummy = jump = ListNode(0)
    dummy.next = l = r = head
    
    while True:
        count = 0
        while r and count < k:   # use r to locate the range
            r = r.next
            count += 1
        if count == k:  # if size k satisfied, reverse the inner linked list
            pre, cur = r, l
            for _ in range(k):
                cur.next, cur, pre = pre, cur.next, cur  # standard reversing
            jump.next, jump, l = pre, l, r  # connect two k-groups
        else:
            return dummy.next